In [19]:
from tensorflow.keras.applications import ResNet101, VGG19, ResNet101V2, ResNet50, ResNet50V2, VGG16, InceptionResNetV2
import tensorflow.keras,os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPool2D , Flatten, Dropout, ZeroPadding2D, GlobalAveragePooling2D, DepthwiseConv2D, Concatenate, add
from tensorflow.keras import optimizers
import numpy as np
from tensorflow.keras.models import load_model
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Model

In [20]:
input_shape=(224, 224, 3)

In [30]:
mm = InceptionResNetV2(include_top=False, weights=None, input_shape=input_shape)

In [29]:
mm.summary()

Model: "inception_resnet_v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_5 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d_203 (Conv2D)             (None, 111, 111, 32) 864         input_5[0][0]                    
__________________________________________________________________________________________________
batch_normalization_203 (BatchN (None, 111, 111, 32) 96          conv2d_203[0][0]                 
__________________________________________________________________________________________________
activation_203 (Activation)     (None, 111, 111, 32) 0           batch_normalization_203[0][0]    
________________________________________________________________________________

In [3]:
# create model
input_img = Input(shape=input_shape, name='main_input')

In [5]:
# model = VGG19(include_top=False, weights=None, input_shape=input_shape)(input_img)
model = VGG16(include_top=False, weights=None, input_shape=input_shape)(input_img)
model.trainable = False

In [6]:
# model1 = ResNet101(include_top=False, weights=None, input_shape=input_shape)(input_img)
model1 = ResNet50(include_top=False, weights=None, input_shape=input_shape)(input_img)
model1.trainable = False

In [ ]:
model1 = ResNet50V2(include_top=True, weights=None, input_shape=input_shape)
model1.trainable = False
model1.summary()

In [7]:
model1 = tf.keras.layers.concatenate([model, model1])

In [8]:
out = Flatten()(model1)
out = Dense(1, activation='sigmoid')(out)

In [9]:
model3 = Model(inputs=input_img, outputs=out)
model3.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
model3.trainable = False
model3.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
vgg16 (Functional)              (None, 7, 7, 512)    14714688    main_input[0][0]                 
__________________________________________________________________________________________________
resnet50 (Functional)           (None, 7, 7, 2048)   23587712    main_input[0][0]                 
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 7, 7, 2560)   0           vgg16[0][0]                      
                                                                 resnet50[0][0]               

In [11]:
model3.save("./tn_model/vgg16_res50.h5")

D:\Program Files\python38\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:494: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  warnings.warn('Custom mask layers require a config and must override '


In [31]:
model2 = Sequential([mm])
model2.add(Flatten())
# model2.add(Dense(units=1024,activation="relu"))
# model2.add(Dense(units=1024,activation="relu"))
model2.add(Dense(units=1, activation="sigmoid"))
model2.compile(optimizer=optimizers.Adam(learning_rate=0.0001), loss='binary_crossentropy', metrics=['accuracy'])

In [32]:
model2.trainable = False
model2.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
inception_resnet_v2 (Functio (None, 5, 5, 1536)        54336736  
_________________________________________________________________
flatten_2 (Flatten)          (None, 38400)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 38401     
Total params: 54,375,137
Trainable params: 0
Non-trainable params: 54,375,137
_________________________________________________________________


In [ ]:
model2.build([train_generator])

In [ ]:
model2.trainable = False
model2.summary()

In [ ]:
model3.summary()

In [16]:
###########################################################
# 데이터 전처리
###########################################################


train_datagen = ImageDataGenerator(rescale=1./255)  # 모든 이미지를 1/255로 스케일을 조정합니다
test_datagen = ImageDataGenerator(rescale=1./255)

train_dir = "./dataset/train2"
validation_dir = './dataset/vali2'

batch_size = 8
# batch_size = 64


train_generator = train_datagen.flow_from_directory( \
        train_dir,                  # 타깃 디렉터리
        target_size=(224, 224),     # 모든 이미지를 224 × 224 크기
        batch_size=batch_size,
        class_mode='binary')        #

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(224, 224),
        batch_size=batch_size,
        class_mode='binary')

Found 802 images belonging to 2 classes.
Found 286 images belonging to 2 classes.


In [17]:
for data_batch, labels_batch in train_generator:
    print('배치 데이터 크기:', data_batch.shape)  #(20, 150, 150, 3)
    print('배치 레이블 크기:', labels_batch.shape) #(20,)
    break

배치 데이터 크기: (8, 224, 224, 3)
배치 레이블 크기: (8,)


In [18]:
steps_per_epochs = 50 # 학습데이터의 총 개수 / 배치 사이즈 (소수점 올림)
epochs = 10

history = model3.fit(
      train_generator ,
      steps_per_epoch=steps_per_epochs  ,
      epochs=epochs ,
      validation_data=validation_generator,
      validation_steps=10)

Epoch 1/10
50/50 [==============================] - 44s 756ms/step - loss: 6.4865 - accuracy: 0.6168 - val_loss: 1.0513 - val_accuracy: 0.5250
Epoch 2/10
50/50 [==============================] - 35s 701ms/step - loss: 2.3164 - accuracy: 0.6100 - val_loss: 1.2485 - val_accuracy: 0.5000
Epoch 3/10
50/50 [==============================] - 35s 705ms/step - loss: 3.6838 - accuracy: 0.6050 - val_loss: 0.9715 - val_accuracy: 0.5000
Epoch 4/10
50/50 [==============================] - 35s 702ms/step - loss: 1.1621 - accuracy: 0.6904 - val_loss: 0.8439 - val_accuracy: 0.5250
Epoch 5/10
50/50 [==============================] - 36s 716ms/step - loss: 3.0325 - accuracy: 0.6375 - val_loss: 242.5414 - val_accuracy: 0.3250
Epoch 6/10
50/50 [==============================] - 36s 715ms/step - loss: 3.0323 - accuracy: 0.6225 - val_loss: 2.4339 - val_accuracy: 0.4500
Epoch 7/10
50/50 [==============================] - 36s 717ms/step - loss: 4.0854 - accuracy: 0.6525 - val_loss: 1582.1996 - val_accuracy: 0

KeyboardInterrupt: 

In [ ]:
###########################################################
# 학습 결과 정확도 및 비용 그래프
###########################################################
import matplotlib.pyplot as plt
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')

plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()

In [ ]:
model2.save("./tn_model/vgg_res_v3.h5")